# Animal Faces

## Задание:

Необходимо реализовать систему классификации лиц животных. Задача представляет из себя классификацию на 3 класса.

1) Необходимо загрузить изображения и привести их к единому формату. Важно понимать, что исходное разрешение очень больше, и простые сети на несколько слоев могут выдать не такое хорошее качество.

2) Необходимо обучить нейросеть для распознавания 3 классов, желательно будет сравнить результаты на нескольких параметрах.

3) Оценить по метрикам качества для каждого класса.

Важно визуализировать результат прогноза (на тесте) в человеко-читаемом формате с подписью.

### Подготовка данных

На этом этапе выполняется подготовка датасета для дальнейшего обучения нейросети.  
Исходные изображения загружаются из Kaggle с помощью kagglehub и сохраняются локально без изменения оригинальной структуры.

Далее выполняется предварительная обработка изображений. Все файлы приводятся к единому формату, цветовой модели RGB и фиксированному размеру. Результат сохраняется в отдельную директорию processed, при этом полностью сохраняется исходная иерархия папок train, val и классов.

Такой подход позволяет не изменять исходные данные, ускоряет дальнейшую загрузку изображений и делает эксперименты воспроизводимыми. Аугментации на этом этапе не применяются и будут добавлены позже при обучении модели.

In [1]:
from scripts.download import download_dataset
from scripts.preprocess import preprocess_dataset


raw_data_dir = download_dataset( dataset_id="andrewmvd/animal-faces", target_dir="data/animal-faces")
processed_data_dir = "data/processed/afhq"

processed_path = preprocess_dataset(
    raw_dir=raw_data_dir,
    processed_dir=processed_data_dir,
    image_size=224,
)

### Dataset downloaded

Files copied to `/home/garret/git/mfti/llm_hw/hw5_pytorch/data/animal-faces`

### Preprocessing dataset

Source directory `/home/garret/git/mfti/llm_hw/hw5_pytorch/data/animal-faces`

Target directory `/home/garret/git/mfti/llm_hw/hw5_pytorch/data/processed/afhq`

Image size `224 x 224`

### Preprocessing finished

Processed images `16130`

Skipped images `0`